see https://betatim.github.io/posts/analysing-the-arxiv/

In [97]:
import urllib
import pandas as pd
import feedparser
import datetime
from bs4 import BeautifulSoup

In [29]:
help(urllib)

Help on package urllib:

NAME
    urllib

MODULE REFERENCE
    https://docs.python.org/3.9/library/urllib
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

PACKAGE CONTENTS
    error
    parse
    request
    response
    robotparser

FILE
    /home/ali/anaconda3/envs/new_torch/lib/python3.9/urllib/__init__.py




In [7]:
import xml.etree.ElementTree as ET
tree = ET.parse('country.xml')
root = tree.getroot()
print(root.tag)

data


In [21]:
for f in dir(root): 
    if '__' not in f: print(f)

append
attrib
clear
extend
find
findall
findtext
get
insert
items
iter
iterfind
itertext
keys
makeelement
remove
set
tag
tail
text


In [10]:
f = open('country.xml').read()
print(f)
r = ET.fromstring(f)
r

<?xml version="1.0"?>
<data>
    <country name="Liechtenstein">
        <rank>1</rank>
        <year>2008</year>
        <gdppc>141100</gdppc>
        <neighbor name="Austria" direction="E"/>
        <neighbor name="Switzerland" direction="W"/>
    </country>
    <country name="Singapore">
        <rank>4</rank>
        <year>2011</year>
        <gdppc>59900</gdppc>
        <neighbor name="Malaysia" direction="N"/>
    </country>
    <country name="Panama">
        <rank>68</rank>
        <year>2011</year>
        <gdppc>13600</gdppc>
        <neighbor name="Costa Rica" direction="W"/>
        <neighbor name="Colombia" direction="E"/>
    </country>
</data>


<Element 'data' at 0x7f580e4c4a40>

In [80]:
OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

start_date='2012-01-01'
end_date='2012-02-05'

category="physics:hep-ex"

def make_url():
    base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
    url = (base_url +
    f"from={start_date}&until={end_date}&" +
    f"metadataPrefix=arXiv&set={category}")
    print(url)
    return url

In [90]:
url=make_url()

http://export.arxiv.org/oai2?verb=ListRecords&from=2012-01-01&until=2012-02-05&metadataPrefix=arXiv&set=physics:hep-ex


you can check this url on your browser (xml schema)

In [95]:
with urllib.request.urlopen(url) as response:
    xml=response.read()
    root = ET.fromstring(xml)
print(root)

<Element '{http://www.openarchives.org/OAI/2.0/}OAI-PMH' at 0x7f57f5734310>


In [122]:
print()
soup=BeautifulSoup(xml).prettify()
# string=soup.string
print(soup.split('<record>')[:2])


['<?xml version="1.0" encoding="UTF-8"?>\n<html>\n <body>\n  <oai-pmh xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd">\n   <responsedate>\n    2023-07-15T10:00:14Z\n   </responsedate>\n   <request from="2012-01-01" metadataprefix="arXiv" set="physics:hep-ex" until="2012-02-05" verb="ListRecords">\n    http://export.arxiv.org/oai2\n   </request>\n   <listrecords>\n    ', '\n     <header>\n      <identifier>\n       oai:arXiv.org:1004.5333\n      </identifier>\n      <datestamp>\n       2012-01-25\n      </datestamp>\n      <setspec>\n       physics:hep-ex\n      </setspec>\n     </header>\n     <metadata>\n      <arxiv xmlns="http://arxiv.org/OAI/arXiv/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://arxiv.org/OAI/arXiv/ http://arxiv.org/OAI/arXiv.xsd">\n       <id>\n        1004.5333\n       </id>

In [68]:


def fetch(arxiv=
    df = pd.DataFrame(columns=("title", "abstract", "categories", "created", "id", "doi"))
    
    while True:
        print("fetching", url)
        try:
            # response = urllib2.urlopen(url)
            response = urllib.request.urlopen(url)
            print('REQUEST successful')
            
        except urllib2.HTTPError as e:
            if e.code == 503:
                to = int(e.hdrs.get("retry-after", 30))
                print("Got 503. Retrying after {0:d} seconds.".format(to))

                time.sleep(to)
                continue
                
            else:
                raise
            
        xml = response.read()

        root = ET.fromstring(xml)
        for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
            arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
            meta = record.find(OAI+'metadata')
            info = meta.find(ARXIV+"arXiv")
            created = info.find(ARXIV+"created").text
            created = datetime.datetime.strptime(created, "%Y-%m-%d")
            categories = info.find(ARXIV+"categories").text

            # if there is more than one DOI use the first one
            # often the second one (if it exists at all) refers
            # to an eratum or similar
            doi = info.find(ARXIV+"doi")
            if doi is not None:
                doi = doi.text.split()[0]

            contents = {'title': info.find(ARXIV+"title").text,
                        'id': info.find(ARXIV+"id").text,#arxiv_id.text[4:],
                        'abstract': info.find(ARXIV+"abstract").text.strip(),
                        'created': created,
                        'categories': categories.split(),
                        'doi': doi,
                        }

            df = pd.DataFrame(contents)

        # The list of articles returned by the API comes in chunks of
        # 1000 articles. The presence of a resumptionToken tells us that
        # there is more to be fetched.
        token = root.find(OAI+'ListRecords').find(OAI+"resumptionToken")
        if token is None or token.text is None:
            break

        else:
            url = base_url + "resumptionToken=%s"%(token.text)

    return df, root, response
    


In [69]:
df, root, response = fetch()

fetching http://export.arxiv.org/oai2?verb=ListRecords&from=2012-01-01&until=2012-02-05&metadataPrefix=arXiv&set=physics:hep-ex
REQUEST successful


In [66]:
df

,title,id,abstract,created,categories,doi
0,Search for the SM Higgs decaying to a b-quark ...,1202.0217,Summary of first ATLAS results is given for th...,2012-02-01,hep-ex,None


In [73]:
response

In [2]:
# Base api query url
base_url_query = 'http://export.arxiv.org/api/query?';


base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"

# Search parameters
search_query = 'all:electron' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 5

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

In [3]:
response = urllib2.urlopen(url)


AttributeError: module 'feedparser' has no attribute '_FeedParserMixin'

In [4]:
response = urllib.urlopen(base_url+query).read()


AttributeError: module 'urllib' has no attribute 'urlopen'